In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler


In [82]:
df = pd.read_csv("/content/drive/MyDrive/dataset_tech/Insurance claims data.csv")
# df = df.dropna(subset=['Total Claim Amount', 'Incident Type']) # Commented out the line causing the error
print(df.shape)
print("Columns in the DataFrame:", df.columns.tolist()) # Print all column names

(58592, 41)
Columns in the DataFrame: ['policy_id', 'subscription_length', 'vehicle_age', 'customer_age', 'region_code', 'region_density', 'segment', 'model', 'fuel_type', 'max_torque', 'max_power', 'engine_type', 'airbags', 'is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors', 'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder', 'transmission_type', 'steering_type', 'turning_radius', 'length', 'width', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert', 'ncap_rating', 'claim_status']


In [83]:



df = df.dropna(subset=['claim_status'])


In [84]:
features = [
    'subscription_length', 'vehicle_age', 'customer_age',
    'airbags', 'is_esc', 'is_adjustable_steering', 'is_parking_sensors',
    'is_parking_camera', 'is_brake_assist', 'is_power_steering',
    'ncap_rating'
]


In [97]:
for col in features:
    if df[col].dtype == 'object':
        df[col] = df[col].map({'Yes': 1, 'No': 0})

In [98]:

df['won'] = 0
np.random.seed(42)
df.loc[df.sample(frac=0.05).index, 'won'] = 1

print("Simulated class distribution:")
print(df['won'].value_counts())


Simulated class distribution:
won
0    55662
1     2930
Name: count, dtype: int64


In [99]:
base_cost = 50000
df['settlement_cost'] = base_cost + df['vehicle_age'] * 1000 + df['customer_age'] * 100
df['litigation_cost'] = df['settlement_cost'] * 1.3

In [100]:
df = df.dropna(subset=features)


X = df[features]
y_settle = df['settlement_cost']
y_litigate = df['litigation_cost']
y_win = df['won']



In [101]:
print(df['won'].value_counts())

won
0    55662
1     2930
Name: count, dtype: int64


In [102]:
print(df['claim_status'].unique())

[0 1]


In [103]:
print(df['won'].value_counts())

won
0    55662
1     2930
Name: count, dtype: int64


In [104]:
print("Claim status values:", df['claim_status'].unique())
print("Row count:", len(df))


Claim status values: [0 1]
Row count: 58592


In [105]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [110]:
X_train, X_test, y_s_train, y_s_test = train_test_split(X_scaled, y_settle, test_size=0.2, random_state=42, stratify=y_win)
_, _, y_l_train, y_l_test = train_test_split(X_scaled, y_litigate, test_size=0.2, random_state=42, stratify=y_win)
X_train_w, X_test_w, y_w_train, y_w_test = train_test_split(X_scaled, y_win, test_size=0.2, random_state=42, stratify=y_win)


In [111]:
settle_model = LinearRegression().fit(X_train, y_s_train)
lit_model = LinearRegression().fit(X_train, y_l_train)
win_model = LogisticRegression(max_iter=1000).fit(X_train_w, y_w_train)


In [113]:
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, classification_report

y_s_pred = settle_model.predict(X_test)
print("\n📊 Settlement Cost Prediction")
print("RMSE:", np.sqrt(mean_squared_error(y_s_test, y_s_pred)))
print("R² Score:", r2_score(y_s_test, y_s_pred))

# Evaluate Litigation Cost Model
y_l_pred = lit_model.predict(X_test)
print("\n📊 Litigation Cost Prediction")
print("RMSE:", np.sqrt(mean_squared_error(y_l_test, y_l_pred)))
print("R² Score:", r2_score(y_l_test, y_l_pred))

# Evaluate Claim Success Classifier
y_w_pred = win_model.predict(X_test_w)
print("\n📊 Win Probability Classifier")
print("Accuracy:", accuracy_score(y_w_test, y_w_pred))
print("Classification Report:\n", classification_report(y_w_test, y_w_pred))


📊 Settlement Cost Prediction
RMSE: 2.645273566604349e-12
R² Score: 1.0

📊 Litigation Cost Prediction
RMSE: 1.8626252813339176e-12
R² Score: 1.0

📊 Win Probability Classifier
Accuracy: 0.9499957334243536
Classification Report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97     11133
           1       0.00      0.00      0.00       586

    accuracy                           0.95     11719
   macro avg       0.47      0.50      0.49     11719
weighted avg       0.90      0.95      0.93     11719



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
